## Problem - "Healthy stocks"
What is the % of symbols from table `bars_1` for which the following condition is satisfied:
`Adj.Close(t) > Average(Adj_close{t-40:t})` for at least one t in 2019?

In [4]:
%run /home/jovyan/notebooks/utils.ipynb

## Solution

In [5]:
execute("""
WITH data AS (
    SELECT symbol, adj_close, AVG(adj_close) OVER (
        PARTITION BY symbol ORDER BY "date" ROWS BETWEEN 40 PRECEDING AND CURRENT ROW
    ) AS historic_adj_close
    FROM bars_1 WHERE EXTRACT(YEAR FROM "date") = 2019
)
SELECT (
    COUNT(DISTINCT symbol)::NUMERIC
    / NULLIF((SELECT COUNT(DISTINCT symbol)::NUMERIC FROM bars_1), 0)
    * 100::NUMERIC
) AS healthy_stocks_pct
FROM data WHERE adj_close > historic_adj_close
""")

,healthy_stocks_pct
0,99.470899


## Considerations
Data from the following years includes one more stock - CARR (first record 2020-03-24 as it had separated and became an independent company)

If we want to correct for this, we have to divide not by the number of all unique symbols but only by the number of unique symbols from 2019

In [6]:
execute("""
WITH data AS (
    SELECT symbol, adj_close, AVG(adj_close) OVER (
        PARTITION BY symbol ORDER BY "date" ROWS BETWEEN 40 PRECEDING AND CURRENT ROW
    ) AS historic_adj_close
    FROM bars_1 WHERE EXTRACT(YEAR FROM "date") = 2019
)
SELECT (
    COUNT(DISTINCT symbol)::NUMERIC
    / NULLIF((
        SELECT COUNT(DISTINCT symbol)::NUMERIC FROM bars_1 WHERE EXTRACT(YEAR FROM "date") = 2019
    ), 0)
    * 100::NUMERIC
) AS healthy_stocks_pct
FROM data WHERE adj_close > historic_adj_close
""")

,healthy_stocks_pct
0,100.0
